<a href="https://colab.research.google.com/github/diegarza/Regresion_No_Lineal/blob/main/Actividad6_A01705984.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

df = pd.read_csv('cuentas_credicel.csv',
                 encoding = 'latin')

<ipython-input-8-bfc84179423b>:3: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cuentas_credicel.csv',


In [9]:
df.columns

Index(['folio', 'tag', 'folio_solicitud', 'fecha', 'marca', 'modelo', 'plazo',
       'precio', 'enganche', 'descuento', 'semana', 'monto_financiado',
       'costo_total', 'monto_accesorios', 'agente_venta', 'dis_venta',
       'status', 'fraude', 'empresa', 'inversion', 'pagos_realizados',
       'reautorizacion', 'fecha_ultimo_pago', 'fecha_pago_proximo',
       'status_cuenta', 'puntos', 'riesgo', 'score_buro', 'razones_buro',
       'porc_eng', 'limite_credito', 'semana_actual', 'cp_cliente',
       'edad_cliente', 'cd_cliente', 'edo_cliente', 'cd_venta', 'edo_venta',
       'curp'],
      dtype='object')

In [10]:
########## Valores nulos #############

df.dropna(inplace=True)

# Columna 'limite_credito' debe corregirse
# Verifica y reemplaza valores no numéricos con la mediana

import re

# Función para extraer números de una cadena con letras y números
def extract_numbers(value):
    numbers = re.findall(r'\d+', str(value))
    if numbers:
        return int(''.join(numbers))
    else:
        return np.nan

In [11]:
df.isnull().sum().sum()

0

In [12]:
######### Elimincación de outliers ############

for col in df.columns:
  if col != 'folio' and col != 'folio_solicitud':
    if df[col].dtypes == 'int64' or df[col].dtypes == 'float64':
      Q1 = df[col].quantile(0.25)
      Q3 = df[col].quantile(0.75)
      IQR = Q3 - Q1
      lower_bound = Q1 - 1.5 * IQR
      upper_bound = Q3 + 1.5 * IQR
      df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

In [ ]:
import numpy as np
import re
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

variables_objetivo = ['riesgo', 'score_buro', 'porc_eng', 'limite_credito']

r2_df = pd.DataFrame(columns=variables_objetivo, index=variables_objetivo)

dataframes = []

# Ciclo for para aplicar los modelos

for i in range(len(variables_objetivo)):
  for j in range(i, len(variables_objetivo)):
    variable_objetivo1 = variables_objetivo[i]
    variable_objetivo2 = variables_objetivo[j]

    if variable_objetivo1 == variable_objetivo2:
      continue

    subset_df = df[[variable_objetivo1, variable_objetivo2]].dropna()

    X = subset_df[[variable_objetivo1]]
    y = subset_df[variable_objetivo2]

    modelos = [
        ("Regresión Polinómica", PolynomialFeatures(degree=2), LinearRegression()),
        ("Random Forest", None, RandomForestRegressor(n_estimators=100, random_state=42)),
        ("SVM", None, SVR(kernel='rbf')),
        ("KNN", None, KNeighborsRegressor(n_neighbors=5)),
        ("Decision Trees", None, DecisionTreeRegressor(random_state=42))
    ]

    r2_df = pd.DataFrame(columns=['Modelo', 'r2'])

    r2_scores = {}

    for nombre_modelo, preprocesamiento, modelo in modelos:
      if preprocesamiento:
        X_preprocesado = preprocesamiento.fit_transform(X)
      else:
        X_preprocesado = X.values.reshape(-1, 1)

      X_preprocesado = X_preprocesado.astype(float)
      y = y.astype(float)

      modelo.fit(X_preprocesado, y)
      y_pred = modelo.predict(X_preprocesado)
      r2 = r2_score(y, y_pred)
      r2_df = r2_df.append({'Modelo': nombre_modelo, 'r2': r2}, ignore_index=True)

      for nombre_modelo, r2_valor in r2_scores.items():
          r2_df.at[variable_objetivo1, f'{variable_objetivo2} - {nombre_modelo}'] = r2_valor
          r2_df.at[variable_objetivo2, f'{variable_objetivo1} - {nombre_modelo}'] = r2_valor

      dataframes.append((variable_objetivo1, variable_objetivo2, r2_df))

In [15]:
# Impresión de los dataframes pertinentes

print(dataframes[4],'\n')
print(dataframes[9],'\n')
print(dataframes[14],'\n')
print(dataframes[19],'\n')
print(dataframes[24],'\n')

('riesgo', 'score_buro',                  Modelo        r2
0  Regresión Polinómica  0.851905
1         Random Forest  0.901987
2                   SVM  0.860811
3                   KNN  0.886945
4        Decision Trees  0.903488) 

('riesgo', 'porc_eng',                  Modelo        r2
0  Regresión Polinómica  0.456987
1         Random Forest  0.668612
2                   SVM  0.500165
3                   KNN  0.615639
4        Decision Trees  0.675185) 

('riesgo', 'limite_credito',                  Modelo        r2
0  Regresión Polinómica  0.619055
1         Random Forest  0.764087
2                   SVM  0.467254
3                   KNN  0.712628
4        Decision Trees  0.772096) 

('score_buro', 'porc_eng',                  Modelo        r2
0  Regresión Polinómica  0.378039
1         Random Forest  0.476445
2                   SVM  0.395084
3                   KNN  0.342422
4        Decision Trees  0.478419) 

('score_buro', 'limite_credito',                  Modelo        r2
0